# Treinamento Nerds Viajantes - LDA

In [17]:
%reload_ext autoreload
%autoreload 2

In [28]:
# Modulos da propria aplicacao
from limpeza import limpeza_posts, limpeza_texto
from repository import nerds_viajantes
from treinamento import dicionario, treinamento_lda

## Trabalho com base de treinamento

In [3]:
# Carregar posts
published = nerds_viajantes.read_published()

In [4]:
# Remove posts desnecessarios
posts = limpeza_posts.limpar_posts(published)

In [5]:
# Limpa texto e gera documentos para treinamento
documentos = posts['content'].apply(limpeza_texto.limpar_texto)

In [27]:
# Cria dicionario com dados da base de treinamento
dicionario_treinamento = dicionario.criar(documentos)
dicionario.salvar_como_texto(dicionario_treinamento)

In [29]:
# Treina modelo com dicionario e documentos
model_lda = treinamento_lda.ajustar_modelo(documentos, dicionario_treinamento)

In [33]:
# Calculo de probabilidades para um documento aleatorio
treinamento_lda.calcular_probabilidades_documento(documentos[20], dicionario_treinamento, model_lda)

[0.00022057103,
 0.0002205708,
 0.00022057042,
 0.00022057176,
 0.00022057089,
 0.00022057076,
 0.97797537,
 0.00022057208,
 0.00022056914,
 0.019598352,
 0.00022057048,
 0.00022057202,
 0.00022057019]